In [35]:
from pyspark.sql import *
import getpass
username = getpass.getuser()
spark = SparkSession.builder \
    .appName('loans_score') \
    .config('spark.ui.port', '0') \
    .config('spark.shuffle.useOldFetchProtocol','true') \
    .config("spark.sql.warehouse.dir", f"/user/{username}/warehouse") \
    .enableHiveSupport() \
    .master('yarn') \
    .getOrCreate()

In [66]:
spark.stop()

# Setting up global variables to ease calculations

In [38]:
spark.conf.set('spark.sql.unacceptable_rated_pts',0)
spark.conf.set('spark.sql.very_bad_rated_pts',100)
spark.conf.set('spark.sql.bad_rated_pts',250)
spark.conf.set('spark.sql.good_rated_pts',500)
spark.conf.set('spark.sql.very_good_rated_pts',650)
spark.conf.set('spark.sql.excellent_rated_pts',800)
spark.conf.set('spark.sql.unacceptable_grade_pts',750)
spark.conf.set('spark.sql.very_bad_grade_pts',1000)
spark.conf.set('spark.sql.bad_grade_pts',1500)
spark.conf.set('spark.sql.good_grade_pts',2000)
spark.conf.set('spark.sql.very_good_grade_pts',2500)

# Loan score = 0.2 * payment_history + 0.45 *default_history + 0.35 * financial health

# Payment History => last payment and what is the total money received 

In [37]:
bad_customer_data_final_df = spark.read.csv('/user/itv012667/lendingclub/bad/bad/bad_customer_data_final',inferSchema=True,header=True)

In [39]:
bad_customer_data_final_df.createOrReplaceTempView("bad_data_customer")

In [40]:
ph_df = spark.sql("select c.member_id, \
   case \
   when p.last_payment < (c.installment * 0.5) then ${spark.sql.very_bad_rated_pts} \
   when p.last_payment >= (c.installment * 0.5) and p.last_payment < c.installment then ${spark.sql.very_bad_rated_pts} \
   when (p.last_payment = (c.installment)) then ${spark.sql.good_rated_pts} \
   when p.last_payment > (c.installment) and p.last_payment <= (c.installment * 1.50) then ${spark.sql.very_good_rated_pts} \
   when p.last_payment > (c.installment * 1.50) then ${spark.sql.excellent_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   end as last_payment_pts, \
   case \
   when p.total_payment >= (c.funded_amount * 0.50) then ${spark.sql.very_good_rated_pts} \
   when p.total_payment < (c.funded_amount * 0.50) and p.total_payment > 0 then ${spark.sql.good_rated_pts} \
   when p.total_payment = 0 or (p.total_payment) is null then ${spark.sql.unacceptable_rated_pts} \
   end as total_payment_pts \
from itv_012667_lending_club.loan_repayments p \
inner join itv_012667_lending_club.loans c on c.loand_id = p.loan_id where member_id NOT IN (select member_id from bad_data_customer)")

In [41]:
spark.sql("select * from itv_012667_lending_club.loans")

loand_id,member_id,loan_amount,funded_amount,term,int_rate,installment,issue_d,loan_status,purpose,title,ingestTimestamp
56633077,b59d80da191f5b573...,3000.0,3000.0,3,7.89,93.86,Aug-2015,Fully Paid,credit_card,Credit card refin...,2024-05-26 20:16:...
55927518,202d9f56ecb7c3bc9...,15600.0,15600.0,3,7.89,488.06,Aug-2015,Fully Paid,credit_card,Credit card refin...,2024-05-26 20:16:...
56473345,e5a140c0922b554b9...,20000.0,20000.0,3,9.17,637.58,Aug-2015,Fully Paid,debt_consolidation,Debt consolidation,2024-05-26 20:16:...
56463188,e12aefc548f750777...,11200.0,11200.0,5,21.99,309.27,Aug-2015,Fully Paid,home_improvement,Home improvement,2024-05-26 20:16:...
56473316,1b3a50d854fbbf97e...,16000.0,16000.0,5,20.99,432.77,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2024-05-26 20:16:...
56663266,1c4329e5f17697127...,20000.0,20000.0,5,13.33,458.45,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2024-05-26 20:16:...
56483027,5026c86ad983175eb...,10000.0,10000.0,3,12.69,335.45,Aug-2015,Fully Paid,other,Other,2024-05-26 20:16:...
56613385,9847d8c1e9d0b2084...,23400.0,23400.0,5,19.19,609.46,Aug-2015,Current,small_business,Business,2024-05-26 20:16:...
56643620,8340dbe1adea41fb4...,16000.0,16000.0,3,5.32,481.84,Jul-2015,Fully Paid,debt_consolidation,Debt consolidation,2024-05-26 20:16:...
56533114,d4de0de3ab7d79ad4...,25450.0,25450.0,3,27.31,1043.24,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2024-05-26 20:16:...


In [42]:
ph_df.createOrReplaceTempView("ph_pts")

In [43]:
ph_df.show()

+--------------------+----------------+-----------------+
|           member_id|last_payment_pts|total_payment_pts|
+--------------------+----------------+-----------------+
|dcec9334e70f1cc95...|             800|              650|
|fc58ca61f51f9dcac...|             500|              650|
|2fb62a6ca51063b11...|             500|              650|
|488268a5531951622...|             800|              650|
|ade6026208e48f5f9...|             500|              650|
|7c8b0ca6acddfaeb1...|             800|              650|
|a707b7fe7c38bad65...|             800|              650|
|1df639cddea30c288...|             800|              650|
|22d67005e12d8726d...|             500|              650|
|009cf312bd46551b4...|             500|              650|
|2d995d383622fa80e...|             500|              650|
|88b97663d2562f239...|             500|              650|
|097d871ba6f21ee37...|             500|              650|
|82e698108e1b8b88a...|             500|              650|
|9e89f004a0147

In [44]:
spark.sql("use itv_012667_lending_club")

""


In [45]:
spark.sql("show tables").show(truncate=False)

+-----------------------+-----------------------------------+-----------+
|database               |tableName                          |isTemporary|
+-----------------------+-----------------------------------+-----------+
|itv_012667_lending_club|customer_data                      |false      |
|itv_012667_lending_club|customers_new                      |false      |
|itv_012667_lending_club|loan_repayments                    |false      |
|itv_012667_lending_club|loans                              |false      |
|itv_012667_lending_club|loans_defaulters_delinq            |false      |
|itv_012667_lending_club|loans_defaulters_delinq_new        |false      |
|itv_012667_lending_club|loans_defaulters_detail_rec_enq    |false      |
|itv_012667_lending_club|loans_defaulters_detail_rec_enq_new|false      |
|                       |bad_data_customer                  |true       |
|                       |ph_pts                             |true       |
+-----------------------+-------------

# Loan Defaulter History
### loan defaulter history => Delinquent in the last 2 years + Public Records + Public Record Bankruptcy filing +Inquiry in the last six months


In [49]:
ldh_ph_df = spark.sql(
    "select p.*, \
    CASE \
    WHEN d.delinq_2yrs = 0 THEN ${spark.sql.excellent_rated_pts} \
    WHEN d.delinq_2yrs BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
    WHEN d.delinq_2yrs BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
    WHEN d.delinq_2yrs > 5 OR d.delinq_2yrs IS NULL THEN ${spark.sql.unacceptable_grade_pts} \
    END AS delinq_pts, \
    CASE \
    WHEN l.pub_rec = 0 THEN ${spark.sql.excellent_rated_pts} \
    WHEN l.pub_rec BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
    WHEN l.pub_rec BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
    WHEN l.pub_rec > 5 OR l.pub_rec IS NULL THEN ${spark.sql.very_bad_rated_pts} \
    END AS public_records_pts, \
    CASE \
    WHEN l.pub_rec_bankruptcies = 0 THEN ${spark.sql.excellent_rated_pts} \
    WHEN l.pub_rec_bankruptcies BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
    WHEN l.pub_rec_bankruptcies BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
    WHEN l.pub_rec_bankruptcies > 5 OR l.pub_rec_bankruptcies IS NULL THEN ${spark.sql.very_bad_rated_pts} \
    END as public_bankruptcies_pts, \
    CASE \
    WHEN l.inq_last_6mths = 0 THEN ${spark.sql.excellent_rated_pts} \
    WHEN l.inq_last_6mths BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
    WHEN l.inq_last_6mths BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
    WHEN l.inq_last_6mths > 5 OR l.inq_last_6mths IS NULL THEN ${spark.sql.unacceptable_rated_pts} \
    END AS enq_pts \
    FROM itv_012667_lending_club.loans_defaulters_detail_rec_enq_new l \
    INNER JOIN itv_012667_lending_club.loans_defaulters_delinq_new d ON d.member_id = l.member_id  \
    INNER JOIN ph_pts p ON p.member_id = l.member_id where l.member_id NOT IN (select member_id from bad_data_customer)")

In [50]:
ldh_ph_df.createOrReplaceTempView("ldh_ph_pts")

# Financial History
### Financial History => Homeownership + Any other loans + Funded Amount (credit limit vs loan acquired) + Grade pts


In [51]:
fh_ldh_ph_df = spark.sql("select ldef.*, \
   CASE \
   WHEN LOWER(l.loan_status) LIKE '%fully paid%' THEN ${spark.sql.excellent_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%current%' THEN ${spark.sql.good_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%in grace period%' THEN ${spark.sql.bad_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%late (16-30 days)%' OR LOWER(l.loan_status) LIKE '%late (31-120 days)%' THEN ${spark.sql.very_bad_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%charged off%' THEN ${spark.sql.unacceptable_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   END AS loan_status_pts, \
   CASE \
   WHEN LOWER(a.home_ownership) LIKE '%own' THEN ${spark.sql.excellent_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%rent' THEN ${spark.sql.good_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%mortgage' THEN ${spark.sql.bad_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%any' OR LOWER(a.home_ownership) IS NULL THEN ${spark.sql.very_bad_rated_pts} \
   END AS home_pts, \
   CASE \
   WHEN l.funded_amount <= (a.total_high_credit_limit * 0.10) THEN ${spark.sql.excellent_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.10) AND l.funded_amount <= (a.total_high_credit_limit * 0.20) THEN ${spark.sql.very_good_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.20) AND l.funded_amount <= (a.total_high_credit_limit * 0.30) THEN ${spark.sql.good_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.30) AND l.funded_amount <= (a.total_high_credit_limit * 0.50) THEN ${spark.sql.bad_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.50) AND l.funded_amount <= (a.total_high_credit_limit * 0.70) THEN ${spark.sql.very_bad_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.70) THEN ${spark.sql.unacceptable_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   END AS credit_limit_pts, \
   CASE \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A1' THEN ${spark.sql.excellent_rated_pts} \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A2' THEN (${spark.sql.excellent_rated_pts} * 0.95) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A3' THEN (${spark.sql.excellent_rated_pts} * 0.90) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A4' THEN (${spark.sql.excellent_rated_pts} * 0.85) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A5' THEN (${spark.sql.excellent_rated_pts} * 0.80) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B1' THEN (${spark.sql.very_good_rated_pts}) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B2' THEN (${spark.sql.very_good_rated_pts} * 0.95) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B3' THEN (${spark.sql.very_good_rated_pts} * 0.90) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B4' THEN (${spark.sql.very_good_rated_pts} * 0.85) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B5' THEN (${spark.sql.very_good_rated_pts} * 0.80) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C1' THEN (${spark.sql.good_rated_pts}) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C2' THEN (${spark.sql.good_rated_pts} * 0.95) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C3' THEN (${spark.sql.good_rated_pts} * 0.90) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C4' THEN (${spark.sql.good_rated_pts} * 0.85) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C5' THEN (${spark.sql.good_rated_pts} * 0.80) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D1' THEN (${spark.sql.bad_rated_pts}) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D2' THEN (${spark.sql.bad_rated_pts} * 0.95) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D3' THEN (${spark.sql.bad_rated_pts} * 0.90) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D4' THEN (${spark.sql.bad_rated_pts} * 0.85) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D5' THEN (${spark.sql.bad_rated_pts} * 0.80) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E1' THEN (${spark.sql.very_bad_rated_pts}) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E2' THEN (${spark.sql.very_bad_rated_pts} * 0.95) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E3' THEN (${spark.sql.very_bad_rated_pts} * 0.90) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E4' THEN (${spark.sql.very_bad_rated_pts} * 0.85) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E5' THEN (${spark.sql.very_bad_rated_pts} * 0.80) \
   WHEN (a.grade) in ('F', 'G') THEN (${spark.sql.unacceptable_rated_pts}) \
   END AS grade_pts \
   FROM ldh_ph_pts ldef \
   INNER JOIN itv_012667_lending_club.loans l ON ldef.member_id = l.member_id \
   INNER JOIN itv_012667_lending_club.customers_new a ON a.member_id = ldef.member_id where ldef.member_id NOT IN (select member_id from bad_data_customer)") 

In [52]:
fh_ldh_ph_df.createOrReplaceTempView("fh_ldh_ph_pts")

# Loan Score

In [53]:
spark.sql("select * ,last_payment_pts+total_payment_pts as paymentHistory, delinq_pts+public_records_pts+public_bankruptcies_pts+enq_pts as default_history, loan_status_pts+home_pts+credit_limit_pts+grade_pts as financialHealth from fh_ldh_ph_pts").createOrReplaceTempView("final_Consolidated")

In [56]:
loan_score = spark.sql("select member_id, paymentHistory,default_history as defaultHistory, financialHealth, 0.2*paymentHistory+0.45*default_history+0.35*financialHealth as loanScore from final_Consolidated")

In [60]:
loan_score.createOrReplaceTempView("loan_score")

# Calcuate the grade

In [63]:
loan_score_final = spark.sql("select ls.*, \
case \
WHEN loanScore > ${spark.sql.very_good_grade_pts} THEN 'A' \
WHEN loanScore <= ${spark.sql.very_good_grade_pts} AND loanScore > ${spark.sql.good_grade_pts} THEN 'B' \
WHEN loanScore <= ${spark.sql.good_grade_pts} AND loanScore > ${spark.sql.bad_grade_pts} THEN 'C' \
WHEN loanScore <= ${spark.sql.bad_grade_pts} AND loanScore  > ${spark.sql.very_bad_grade_pts} THEN 'D' \
WHEN loanScore <= ${spark.sql.very_bad_grade_pts} AND loanScore > ${spark.sql.unacceptable_grade_pts} THEN 'E'  \
WHEN loanScore <= ${spark.sql.unacceptable_grade_pts} THEN 'F' \
end as loan_final_grade \
from loan_score ls")

In [64]:
loan_score_final

member_id,paymentHistory,defaultHistory,financialHealth,loanScore,loan_final_grade
000c8875b71a6b47c...,1450,2650,2530.00,2368.0000,B
003769d7f54c7859e...,1000,2650,1150.00,1795.0000,C
003e1e6cbd2920bbb...,1150,1550,2190.00,1694.0000,C
004017b21bd4d6271...,750,3150,2350.00,2390.0000,B
005b4c3db3fce07dc...,1150,1550,1770.00,1547.0000,C
00710707c563c2119...,1450,2650,2370.00,2312.0000,B
007da79904f69970d...,1450,2650,2350.00,2305.0000,B
00f435a80d0440ece...,1000,2500,2025.00,2033.7500,B
00fc2ae3ffb1213e4...,1300,2650,2150.00,2205.0000,B
00fc8144cb210ba8c...,1150,1550,2050.00,1645.0000,C


In [65]:
loan_score_final.write.mode("overwrite").option("path","/user/itv012667/lendingclub/processed/loan_score").save()